In [2]:
# !pip install torchinfo

In [3]:
from typing import Optional

import math
from tqdm import tqdm
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.init import xavier_uniform_
from torch.utils.data import Dataset, DataLoader

from torchinfo import summary

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
@dataclass
class ModelConfig:
    d_model: int = 512
    n_heads: int = 8
    n_layers: int = 8
    d_ff: int = 2048
    attn_dropout: float = 0.1
    mlp_dropout: float = 0.1
    qkv_bias: bool = False

    max_len: int = 2048 # context_len, seq_len
    src_vocab_size: int = 1024
    tgt_vocab_size: int = 1024

# Transzformer

Az architektúra egy kódoló és dekódoló részből áll. A kódoló az $\mathbf{x} = ( x_1, x_2, \ldots, x_n )$ bemeneti szekvenciát a $\mathbf{z} = (z_1, z_2, \ldots, z_n)$ folytonos reprezentációvá képezi le, majd a dekódoló a $\mathbf{z}$ reprezentációból és az $\mathbf{x}$ szekvenciából az $\mathbf{y} = (y_1, y_2, \ldots, y_m)$ kimeneti szekvenciát generálja, ahol $n,m \in \mathbb{N}^+$ .

Minden egyes (idő)lépésben, a kimenet következő elemének generálásához a korábbi kimeneti elemek is hozzájárulnak, ezért a modell **auto-regresszív** (auto-regressive).

A kódoló és dekódoló komponensek egymásra épülő rétegek sorozata, amelyek mindegyike figyelem (attention) mechanizmusból és teljesen összekapcsolt előrecsatolt hálózatból (fully connected feed-forward network) áll.

Megjegyzés. A bemeneti szekvencia a tokenizálás után előálló sorozat, ahol a szöveg ún. tokenekre van bontva, ami lehet szó, szórész, karakter (vagy bájt). A tokenek egyedi azonosítók, $\mathbb{N}$ elemei.

In [6]:
class Transformer(nn.Module):

    def __init__(self, config: ModelConfig):
        super().__init__()

        self.src_embedding = Embeddings(config.src_vocab_size, config.d_model)
        self.tgt_embedding = Embeddings(config.tgt_vocab_size, config.d_model)

        self.pe = PositionalEncoding(config)

        self.encoder = Encoder(config)
        self.decoder = Decoder(config)

        self.head = nn.Linear(config.d_model, config.tgt_vocab_size)

    def forward(self, src, tgt) -> torch.Tensor:

        tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt)

        memory = self.encode(src, src_padding_mask)
        logits = self.decode(tgt, memory, tgt_mask, tgt_padding_mask)

        return logits

    def encode(self, src, padding_mask):
        src = self.pe(self.src_embedding(src))
        memory = self.encoder(src, attn_mask = None, padding_mask = padding_mask)
        return memory

    def decode(self, tgt, memory, attn_mask, padding_mask):
        tgt = self.pe(self.tgt_embedding(tgt))
        x = self.decoder(tgt, memory, attn_mask, padding_mask)
        logits = self.head(x)
        return logits

    def _reset_parameters(self):
        for p in self.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

# Kódoló

Az eredeti implementációban a kódoló hat egymásra épülő identikus rétegből áll. Minden réteg két alrétegből áll.

Az első a többszörös vagy többfejű figyelem (multihead attention) mechanizmus, a második egy teljesen összekapcsolt előrecsatolt hálózat.

A szerzők reziduális kapcsolatokat (residual connection, skip connection) alkalmaztak mindkét alréteg körül. A reziduális kapcsolatok közvetlenül biztosítják a gradiensek áramlását a hálózaton keresztül azáltal, hogy egy vagy több réteget átugranak.

Utána, réteg normalizálást (layer normalization) alkalmaztak, hogy stabilizálják a tanulási folyamatot. Azaz minden alréteg kimenete a következőképpen számolható:

$$\text{LayerNorm}(x + \text{Sublayer}(x)),$$

ahol a $\text{Sublayer}(x)$ az alréteg műveletét jelenti. Minden modellbeli alréteg, beleértve a beágyazás réteget, 512 dimenziós kimenetet állít elő.

In [7]:
class Encoder(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()

        self.layers = nn.Sequential(
            *[EncoderBlock(config) for _ in range(config.n_layers)]
        )

    def forward(self, x, attn_mask = None, padding_mask = None):
        for layer in self.layers:
            x = layer(x, attn_mask, padding_mask)
        return x

In [8]:
class EncoderBlock(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()

        self.multi_head_attention = MultiHeadAttention(
            config.d_model,
            config.n_heads,
            config.attn_dropout
        )
        self.layer_norm1 = LayerNorm(config.d_model)

        self.ffd = PositionwiseFeedForward(config)
        self.layer_norm2 = LayerNorm(config.d_model)

    def forward(self, x, attn_mask = None, padding_mask = None) -> torch.Tensor:
        x = self.layer_norm1(
            x + self.multi_head_attention(x, x, x, attn_mask=None, padding_mask=padding_mask)
        )
        x = self.layer_norm2(x + self.ffd(x))
        return x

In [9]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6) -> None:
        super().__init__()

        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

# Dekódoló

A dekóder szintén hat egymásra épülő, azonos felépítésű rétegből áll és minden réteg három alréteget tartalmaz.

Az első alréteg egy maszkolt többfejű figyelem (masked multi-head attention) mechanizmus, amely biztosítja, hogy az adott pozícióhoz tartozó kimenet csak a saját és a megelőző pozíciókra támaszkodjon.

Továbbá, a kimeneti beágyazások egy pozícióval eltolva kerülnek a hálózatba, ami a maszkolással együtt garantálja, hogy az $i$-dik pozícióhoz tartozó predikció csak az $i$-nél kisebb pozíciók kimeneteitől függjön.

A második és harmadik alréteg a kódoló alrétegekkel megegyezik, viszont a második többfejű figyelem a kódoló kimenetét is figyelembe veszi.

A kódolóhoz hasonlóan minden alréteg köré reziduális kapcsolatot építettek, majd réteg normalizálást alkalmaztak.

In [10]:
class Decoder(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()

        self.layers = nn.Sequential(*[DecoderBlock(config) for _ in range(config.n_layers)])

    def forward(self, x, memory, attn_mask, padding_mask = None):
        for layer in self.layers:
            x = layer(x, memory, attn_mask, padding_mask)
        return x

In [11]:
class DecoderBlock(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()

        self.masked_mha = MultiHeadAttention(
            config.d_model,
            config.n_heads,
            config.attn_dropout,
            config.qkv_bias
        )
        self.layer_norm1 = LayerNorm(config.d_model)

        self.mha = MultiHeadAttention(
            config.d_model,
            config.n_heads,
            config.attn_dropout,
            config.qkv_bias
        )
        self.layer_norm2 = LayerNorm(config.d_model)

        self.ffd = PositionwiseFeedForward(config)
        self.layer_norm3 = LayerNorm(config.d_model)

        self.dropout = nn.Dropout(config.mlp_dropout)

    def forward(self, x, memory, attn_mask, padding_mask = None):
        x = self.layer_norm1(x + self.masked_mha(x, x, x, attn_mask, padding_mask))
        x = self.layer_norm2(x + self.mha(x, memory, memory, attn_mask = None, padding_mask = padding_mask))
        x = self.layer_norm3(x + self.ffd(x))
        return x

# Beágyazási réteg

A beágyazási réteg egy tanítható réteg, amely a diszkrét bemenetet (tokeneket) folytonos $d$ dimneziós vektorokká alakítja. Egy egyszerű keresőtábla, amely rögzített szótárhoz (vocab) tárol beágyazásokat, vagyis vektortérbeli reprezentációkat. A szótár minden egyes eleméhez egyedi vektor tartozik.

A beágyazási rétegekben, a súlyok $\sqrt{d}$-vel vannak skálázva és a dimenziója megegyezik a modell belső rétegeinek dimenziójával, vagyis $d$-vel. Ha a bemeneti szekvencia maximális hossza $n$, akkor $n$ új beágyazásvektort tanulunk meg - minden token pozícióra egyet.

Megjegyzés. A szótár angolul könnyen összetéveszhető a dictionary adatszerkezettel (magyarul hash tábla), de a mélytanulás terminológia során a modell szókészletét, szókincset jelenti.

In [12]:
class Embeddings(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()

        self.emb = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        x = self.emb(x)
        return x * math.sqrt(self.d_model)

# Pozicionális kódolás

A transzformer architektúra explicit módon nem tudja megragadni a tokenek sorrendjét a szekvenciában, mivel a kontextust csak a figyelem mechanizmuson keresztül modellezi, ami nem veszi figyelembe a szavak sorrendjét. Ezért, a sorrend megragadására a transzformer egy pozicionális kódolás nevű technikát alkalmaz, amely során a szekvencia elemei, a tokenek relatív vagy abszolút pozíciójáról kapott információt beépíti a szekvenciába.

Ez azt jelenti, hogy a pozícióra vonatkozó infromációt adnak hozzá a bemeneti beágyazásokhoz a kódoló és dekódoló komponensekben, és ugyanaolyan $d$ dimenzióval rendelkezik, mint a beágyazási réteg, az összeadás művelete érdekében. A kódoló és dekódoló bemenete a $t$ pozícióban levő szóbeágyazás és a $t$ pozícióhoz tartozó pozicionális beágyazás összege. Többféle tanítható és rögzített pozíció kódolás létezik, de a szerzők egy rögzített pozíció kódolást alkalmaztak, amely a következőképpen számolható:

$$
\begin{aligned}
P E_{(t, 2 i)} & =\sin \left(t / 10000^{2 i / d}\right) \\
P E_{(t, 2 i+1)} & =\cos \left(t / 10000^{2 i / d}\right)
\end{aligned}
$$

ahol $t$ a pozíció, $i$ a dimenzió indexe és $d$ a modell dimenziója. A pozícionális kódolás minden egyes dimenziója egy szinuszoid függvény. A hullámhosszúságok $2 \pi$-től $10000 \cdot 2 \pi$-ig terjednek, és a dimenziók páros és páratlan indexei szinusz és koszinusz függvények.

In [13]:
class PositionalEncoding(nn.Module):
    def __init__(self, config: ModelConfig) -> None:
        super().__init__()

        pe = torch.zeros(config.max_len, config.d_model)

        position = torch.arange(0, config.max_len).unsqueeze(1)
        div_term = 1 / 10000.0 ** (torch.arange(0, config.d_model, 2) / config.d_model)

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)

        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return x

# Kimeneti réteg

A modell kimeneti rétege egy lineáris rétegből és egy softmax függvényből áll, amely a dekódoló utolsó rétegének kimenetét egy diszkrét valószínűségi eloszlássá alakítja, amely megadja a következő token valószínűségét a szótár minden elemére vonatkozóan.

A két beágyazási réteg súlya össze van kapcsolva egymással és a lineáris rétegben használt súllyal, amit súlykötésnek (weight tying) neveznek. A technika javítja a nyelvi modellek teljesítményét és jelentősen csökkenti a szükséges paraméterek számát.

A beágyazás rétegek megtanulják a szavak reprezentációját, így a hasonló jelentésű tokenek vektorai közel helyezkednek el egymáshoz.

Press és Wolf kimutatták, hogy az utolsó lineáris réteg súlymátrixa, amelyben minden tokennnek van egy vektor reprezentációja, szintén megjeleníti ezt a tulajdonságot. Ezért javasolták a beágyazási réteg és az utolsó lineáris réteg mátrixainak megosztását, amelyet ma szinte minden nyelvi modell alkalmaz.


A kimeneti réteg a `Transformer` osztály kódját leíró cella 14. sorában található.

# Előrecsatolt hálózat

A kódoló és a dekódoló minden rétege tartalmaz egy teljesen összekapcsolt előrecsatolt hálózatot, amelyet minden pozícióra külön-külön, de azonos módon alkalmaz. A hálózat két lineáris leképezésből áll, amelyek között egy ReLU aktivációs függvény helyezkedik el:

$$
\begin{align}
\operatorname{FFN}(x) = \max \left(0,\, \mathbf{x} \mathbf{W}_1 + \mathbf{b}_1 \right) \mathbf{W}_2 + \mathbf{b}_2
\end{align}
$$

ahol $\mathbf{W}_1 \in \mathbb{R}^{d \times d_{ff} }, \mathbf{W}_2 \in \mathbb{R}^{d_{ff} \times d }$ a súlymátrixok, $\mathbf{b}_1 \in \mathbb{R}^{d_{ff}}, \mathbf{b}_2 \in \mathbb{R}^{d}$ az eltolásvektorok és $\max (0, \cdot)$ a ReLU aktivációs függvény.

A lineáris leképezések minden pozíció esetében azonosak, ugyanakkor a különböző rétegek eltérő paramétereket használnak. Másképpen megfogalmazva, ez a szerkezet két, egydimenziós konvolúciónak is tekinthető, ahol a kernel méret 1. A bemenet és a kimenet dimenziója $d = 512$, míg a belső réteg dimenziója $d_{ff} = 2048$.


In [14]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()

        self.ff = nn.Sequential(
            nn.Linear(config.d_model, 4 * config.d_model),
            nn.ReLU(),
            nn.Linear(4 * config.d_model, config.d_model),
            nn.Dropout(config.mlp_dropout),
        )

    def forward(self, x):
        return self.ff(x)

# Figyelem

A figyelem mechanizmus lehetővé teszi a modell számára, hogy a rejtett állapotokból (hidden state) álló szekvencia minden pozíciója kölcsönhatásba lépjen ugyanabban a szekvenciában lévő többi pozícióval, ezáltal az egyes szekvenciákon belül képes megragadni a távolsági (és közeli) kontextust is.

Legyen $d$ beágyazás dimenzió és $\mathbf{h}_t \in \mathbb{R}^d$ a bemeneti szekvencia $t$-edik elemének beágyazása egy figyelem rétegnél. A figyelem mechanizmus a bemenetet először három eltérő súlyvektor segítségével három különböző reprezentációra, ún. lekérdezés-, kulcs- és értékvektorokra képezi le, de a gyakorlatban a számításokat mátrixokba rendezik a hatékonyság érdekében. A $\mathbf{W}^Q, \mathbf{W}^K \in \mathbb{R}^{d \times d_k}$, $\mathbf{W}^V \in \mathbb{R}^{d \times d_v}$ súlymátrixok állítják elő a $\mathbf{q}_t, \mathbf{k}_t \in \mathbb{R}^{d_k}$ és $\mathbf{v}_t \in \mathbb{R}^{d_v}$ reprezentációkat:

$$
\begin{aligned}
& \mathbf{q}_t=\mathbf{W}^Q \mathbf{h}_t, \\
& \mathbf{k}_t=\mathbf{W}^K \mathbf{h}_t, \\
& \mathbf{v}_t=\mathbf{W}^V \mathbf{h}_t,
\end{aligned}
$$

A $\mathbf{q}$ lekérdezés, ahonnan a figyelem irányul, mint a cél a figyelem mechanizmusban. A $\mathbf{k}$ kulcs, amire a figyelem irányul, mint a forrás az egyszerű figyelem mechanizmus esetén. A $\mathbf{v}$ érték az éppen generált kontextus. A képletben szereplő $q,k, v$ betűk az angol query, key, value szavak kezdőbetűi.

Következőnek, a lekérdezés- és kulcs skalárszorzatát veszik és elosztják a $\sqrt{d_k}$ skálázási faktorral a numerikus stabilitás érdekében. Ezután, softmax függvényt alkalmazva a figyelem súlyokat kapják meg, amik az értékkel kerülnek súlyozásra. Ezáltal a $t$-edik token reprezentációjának (vagy kódolt kimenetének) eredménye kiszámítható a figyelem mechanizmus alkalmazásával az alábbi módon:

$$
\mathbf{o}_{t}= \operatorname{softmax} \left( \frac{\mathbf{q}_{t} \mathbf{k}_{t}^T}{\sqrt{d_k}}\right) \mathbf{v}_{t}
$$

A szekvencia tokenjeinek kódolt kimenetei egyszerre számíthatók, mivel a fenti egyenletek kifejezhetőek olyan mátrixműveletekkel, amelyek hatékony módon számíthatóak párhuzamos számításokra specializált, modern hardvereken. A fenti egyenletet \textbf{skálázott skalárszorzat-alapú figyelem}nek (scaled dot-product attention) nevezzük.

Megjegyzés. A $d$, $d_k$ és $d_v$ gyakran megegyezik.

In [15]:
class Head(nn.Module):
    def __init__(self, d_model: int, d_out: int, attn_dropout: float = 0.1, qkv_bias: bool = False) -> None:
        super().__init__()

        self.key = nn.Linear(d_model, d_out, bias = qkv_bias)
        self.query = nn.Linear(d_model, d_out, bias = qkv_bias)
        self.value = nn.Linear(d_model, d_out, bias = qkv_bias)

        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, query, key, value, attn_mask = None, padding_mask = None) -> torch.Tensor:

        b, t, d_model = query.shape

        q = self.query(query)   # (b, t, d_k)
        k = self.key(key)       # (b, t, d_k)
        v = self.value(value)   # (b, t, d_v)

        # (b, t, t) = (b, t, d) @ (b, d, t)
        attn_scores = q @ k.transpose(1, 2)

        # figyelem és padding maszk összefűzése
        if padding_mask is not None:
            padding_mask = padding_mask.view(b, 1, t)
            if attn_mask is None:
                attn_mask = padding_mask
            else:
                attn_mask = attn_mask + padding_mask # := logikai és

        if attn_mask is not None:
            attn_scores.masked_fill_(attn_mask, -torch.inf)

        attn_weights = torch.softmax(attn_scores / k.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ v
        return context_vec

# Többfejű figyelem

A figyelem függvény egyszeri alkalamzása helyett a szerzők előnyösnek találták, hogy a lekérdezéseket, kulcsokat és értékeket $h$ alkalommal, különböző tanulható lineáris leképezésekkel vetítsék le rendre $d_k$ és $d_v$ dimenziókra. Az így kapott projektált lekérdezések, kulcsok és értékek mindegyikén párhuzamosan kerül kiszámításra a figyelemfüggvény, amely \(d_v\) dimenziós kimeneteket eredményez. A kimeneteket konkatenálják és egy utolsó lineáris leképezést alkalmaznak, hogy az eredeti $d$ dimenziós kimenet álljon elő. Ezt a technikát többszörös vagy \textbf{többfejű figyelem}nek (multi-head attention) nevezzük.


A többfejű figyelem lehetővé teszi, hogy a modell különböző pozíciókban a reprezentáció különböző altereiből származó információkra egyidejűleg "figyeljen", gazdagabb kontextuális információt megragadva. Egyetlen figyelemfej esetén az átlagolás ezt a képességet korlátozná, mivel az információk összemosódnak.

Formálisan, legyen $h$ a figyelemfejek száma és $d_h$ a figyelemfejek dimenziója. A fentebbi egyenletben szereplő tagok a következőképpen alakulnak: a $\mathbf{q}_t, \mathbf{k}_t, \mathbf{v}_t \in \mathbb{R}^{d_h h}$ reprezentációk előállnak a $\mathbf{W}^Q, \mathbf{W}^K, \mathbf{W}^V \in \mathbb{R}^{d_h h \times d}$ súlymátrixokból. Azután, a $\mathbf{q}_t, \mathbf{k}_t, \mathbf{v}_t$ vektorokat $h$ részre osztjuk a többfejű figyelem kiszámításához, minden egyes figyelemfejhez egy-egy lekérdezés, kulcs és érték tartozik. Minden figyelemfejre külön-külön alkalmazzuk a skálázott skalárszorzat-alapú figyelem függvényt, majd a kimeneteket konkatenáljuk és egy utolsó lineáris rétegen keresztül leképezzük az eredeti $d$ dimenzióra:

$$
\begin{aligned}
& {\left[\mathbf{q}_{t, 1} ; \mathbf{q}_{t, 2} ; \ldots ; \mathbf{q}_{t, h}\right]=\mathbf{q}_t,} \\
& {\left[\mathbf{k}_{t, 1} ; \mathbf{k}_{t, 2} ; \ldots ; \mathbf{k}_{t, h}\right]=\mathbf{k}_t,} \\
& {\left[\mathbf{v}_{t, 1} ; \mathbf{v}_{t, 2} ; \ldots ; \mathbf{v}_{t, h}\right]=\mathbf{v}_t,}
\end{aligned}
$$

$$
\mathbf{o}_{t, i}=\sum_{j=1}^t \operatorname{Softmax}_j\left(\frac{\mathbf{q}_{t, i}^{\top} \mathbf{k}_{j, i}}{\sqrt{d_h}}\right) \mathbf{v}_{j, i},
$$

$$
\mathbf{u}_t=\mathbf{W}^O\left[\mathbf{o}_{t, 1} ; \mathbf{o}_{t, 2} ; \ldots ; \mathbf{o}_{t, h}\right].
$$

A szerzők a tanulmányban $h = 8$ párhuzamos figyelemréteget, azaz fejet alkalmaztak. Mindegyik fej esetében $d_k = d_v = d / h = 64$ dimenziót használtak. Mivel minden fej csökkentett dimenziójú térben működik, a teljes számítási költség megközelítőleg megegyezik az egyetlen fejjel végzett, teljes dimenziójú figyelem számítási költségével.

A transzformer architektúra *háromféle módon alkalmazza a többfejű figyelmet*.

Először, a "kódoló-dekódoló figyelem" rétegekben a lekérdezés a dekóder előző rétegéből, míg a kulcs és az érték a kódoló kimenetéből származik. Ez lehetővé teszi, hogy a dekódoló minden pozíciója a bemeneti szekvencia összes pozícióját számításba vegye.

Másodszor, a kódolóban a figyelem rétegekben a kulcsok, az értékek és a lekérdezések mind ugyanabból a forrásból származnak, azaz a kódoló előző rétegének kimenetéből. Ezt **önfigyelem**nek (self-attention) nevezzük. Ezáltal a kódoló minden pozíciója képes a kódoló előző rétegének bármely pozíciójára figyelni.

Harmadszor, a dekódoló önfigyelem rétegei lehetővé teszik, hogy a dekódolóban minden pozíció észrevegye a korábbi, illetve az aktuális pozícióit. Azonban ebben az esetben meg kell akadályozni az információ balról jobbra történő áramlását, hogy az adott token ne függjön az utána következő tokenektől és csak az előtte lévő tokeneket tudja számításba venni, ezáltal megőrizve az autoregresszív tulajdonságot. A skálázott skalárszorzat-alapú figyelem során ez úgy kerül megvalósításra, hogy softmax függvény bemeneténél a jövőbeli pozícióknak megfelelő értékeket $-\infty$-re állítjuk, aminek hatására a softmax kimenetében ezek nulla értéket vesznek fel, és így nem befolyásolják figyelem számítást. Ezt a módszert **maszkolás**nak (masking), illetve **kauzális maszk**nak (causal mask) is szokták nevezni és kritikus a nyelvi modellek esetében ahol a következő tokent a korábbi tokenek segítségével generáljuk. Az önfigyelem mechanizmus kauzális maszkolással kiegészített változatát **kauzális figyelem**nek (causal attention) is nevezik.

In [16]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, h: int, attn_dropout: float = 0.1, qkv_bias: bool = False):
        super().__init__()

        assert d_model % h == 0, "d_model is indivisible by h (n_heads)"

        self.d_out = d_model // h # = d_k = d_v

        self.heads = nn.ModuleList(
            [ Head(d_model, self.d_out, attn_dropout, qkv_bias) for _ in range(h) ]
        )
        self.w_o = nn.Linear(h * self.d_out, d_model) # out_proj

    def forward(self, query, key, value, attn_mask = None, padding_mask = None):
        out = torch.cat([head(query, key, value, attn_mask, padding_mask) for head in self.heads], dim=-1)
        out = self.w_o(out)
        return out

In [17]:
def create_mask(src, tgt, padding_token: int = 1024) -> tuple:

    device = src.device
    tgt_seq_len = tgt.shape[1]

    # kauzális maszk, ebben a lépésben még nem állítjuk
    # -inf-re az értékek, csak a figyelem osztály metódusában
    tgt_mask = (torch.triu(torch.ones((tgt_seq_len, tgt_seq_len), device=device)) == 0).transpose(0,1)

    # padding maszk, ahol az igaz érték := maszkolás
    src_padding_mask = (src == padding_token)
    tgt_padding_mask = (tgt == padding_token)

    return tgt_mask, src_padding_mask, tgt_padding_mask

# Futtatás

In [18]:
config = ModelConfig()

In [19]:
model = Transformer(config)

In [20]:
model = model.to(device)

In [21]:
summary(model)

Layer (type:depth-idx)                             Param #
Transformer                                        --
├─Embeddings: 1-1                                  --
│    └─Embedding: 2-1                              524,288
├─Embeddings: 1-2                                  --
│    └─Embedding: 2-2                              524,288
├─PositionalEncoding: 1-3                          --
├─Encoder: 1-4                                     --
│    └─Sequential: 2-3                             --
│    │    └─EncoderBlock: 3-1                      3,150,848
│    │    └─EncoderBlock: 3-2                      3,150,848
│    │    └─EncoderBlock: 3-3                      3,150,848
│    │    └─EncoderBlock: 3-4                      3,150,848
│    │    └─EncoderBlock: 3-5                      3,150,848
│    │    └─EncoderBlock: 3-6                      3,150,848
│    │    └─EncoderBlock: 3-7                      3,150,848
│    │    └─EncoderBlock: 3-8                      3,150,848
├─Decoder: 

A teszteléshez hozzunk létre egy forrás szekvencia és cél szekvencia tenzort, aminek mérete a batch-méret és szekvencia hossza.

Természetesen, a forrásnyelv szövege nem feltétlen egyenlő a célnyelv szövegével, ezért ún. padding-et alkalmazunk, aminek lényege, hogy egy (padding) tokent adunk hozzá a szekvenciához, hogy két tenzor mérete megegezzen.

A padding tokenek nem kerülnek bele a figyelem számításába.

Az egyszerűség kedvéért tegyük fel, hogy a forrás és cél szekvenciának 10 eleme (tokenje) van és a batch-méret 2.

A képzeletbeli szótár (`vocab_size`) jelen esetben 1024 szavas. Továbbá, legyen a padding token értéke 1024.

In [22]:
batch_size = 2
vocab_size = 1024
seq_len = 10

In [23]:
src = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
tgt = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)

In [24]:
assert src.shape == tgt.shape, "source and target tensors are not equal"

In [25]:
results = model(src, tgt)

In [26]:
assert results.shape == (batch_size, seq_len, vocab_size), "results tensor shape is not correct"

In [27]:
results

tensor([[[-0.7474,  0.6309,  1.3059,  ..., -0.3122, -0.5644,  1.3268],
         [-0.4822,  0.6127,  0.3049,  ..., -0.3428, -0.3646,  0.7711],
         [ 0.5773,  0.5921,  1.2765,  ..., -0.0479, -0.2274,  1.0196],
         ...,
         [ 0.3785,  0.7820,  0.2595,  ..., -0.3524, -0.4440,  1.5753],
         [-0.0929,  0.9529,  1.1653,  ...,  0.1805, -0.0149,  0.5563],
         [ 0.1143,  0.2492,  0.3625,  ..., -0.4894,  0.1405,  0.7082]],

        [[ 0.0780,  0.2454,  0.6327,  ..., -0.4938, -0.6576,  1.0808],
         [-0.3486, -0.2542,  0.8601,  ..., -0.4373, -0.4138,  0.4963],
         [-0.2479, -0.0816,  1.2530,  ..., -0.1385, -0.5231,  0.9671],
         ...,
         [-0.3881,  0.3099, -0.0347,  ..., -0.3726,  0.4479,  1.3677],
         [-0.2242,  0.1946,  0.5817,  ..., -0.2176,  0.1519,  0.4154],
         [-0.1084,  0.3643,  0.6808,  ...,  0.2020, -0.2341,  0.9754]]],
       device='cuda:0', grad_fn=<ViewBackward0>)

Az eredmény egy tenzor, aminek mérete a batch-méret, szekvencia hossza és a szekvencia minden eleméhez egy valószínűségi eloszlás (am), ami megadja az adott pozícióra prediktált következő token teljes szókészletre kiterjedő valószínűségeit (logitjeit).